# Retrieval Augmented Generation (RAG) and Vector Databases

In [1]:
!pip install getenv openai

In [2]:
import os
import pandas as pd
import numpy as np
import openai

C:\Users\bethanycheum\AppData\Local\Temp\ipykernel_24812\2781619465.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Creating our Knowledge base

Creating a Azure Cosmos DB database


In [3]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [4]:
## create your cosmoss db on Azure CLI using the following commands
## az login
## az group create -n <resource-group-name> -l <location>
## az cosmosdb create -n <cosmos-db-name> -r <resource-group-name>
## az cosmosdb list-keys -n <cosmos-db-name> -g <resource-group-name>


In [5]:
# chunk our data into 5MB chunks
import json
from azure.cosmos import CosmosClient, PartitionKey, exceptions

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Initialize Cosmos Client
url = os.getenv('COSMOS_DB_ENDPOINT')
key = os.getenv('COSMOS_DB_KEY')
client = CosmosClient(url, credential=key)

# Select database
database_name = 'rag-cosmos-db'
database = client.get_database_client(database_name)

# Select container
container_name = 'data'
container = database.get_container_client(container_name)



In [9]:
# splitting our data into chunks
document_chunks = ["data/frameworks.md", "data/own_framework.md", "data/perceptron.md"]

def split_documents(documents, chunk_size=1000, chunk_overlap=0):
    for document in documents:
        for i in range(0, len(document), chunk_size - chunk_overlap):
            chunk = document[i:i + chunk_size]
            document_chunks.append(chunk)
    return document_chunks


document_chunks = split_documents(document_chunks)
document_chunks

## Converting our text to embeddings

Converting out text  to embeddings, and storing them in our database in chunks

In [ ]:
import requests

API_KEY = os.getenv("AZURE_OPENAI_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-07-01-preview"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": API_KEY})

: 

In [ ]:
from openai.embeddings_utils import get_embedding, cosine_similarity

def create_embeddings(document_chunks, engine="text-embedding-ada-002"):
    # Create embeddings for each document chunk
    embeddings = [get_embedding(chunk, engine) for chunk in document_chunks]
    return embeddings

# Assuming document_chunks is your list of document chunks
data_chunks = create_embeddings(document_chunks)

: 

In [ ]:
data_chunks

[[-0.006301294546574354,
  0.010888524353504181,
  -0.00509301945567131,
  -0.04838719591498375,
  0.0035721384920179844,
  0.01742163859307766,
  -0.035096172243356705,
  -0.015496828593313694,
  0.003969042561948299,
  -0.009153385646641254,
  0.01204762514680624,
  -0.008507099002599716,
  -0.003765322035178542,
  0.004165738821029663,
  -0.01257448922842741,
  0.01743568852543831,
  -0.0005229125963523984,
  0.01795552857220173,
  0.004165738821029663,
  -0.03273582085967064,
  0.00871081929653883,
  -0.012293495237827301,
  0.01265878789126873,
  -0.024235747754573822,
  -0.01503318827599287,
  0.005658520385622978,
  0.0035756509751081467,
  -0.021186960861086845,
  -0.017000148072838783,
  -0.005335377063602209,
  0.05066325142979622,
  -0.016957998275756836,
  -0.006722785532474518,
  -0.012799284420907497,
  0.003954993095248938,
  -0.005813067313283682,
  -0.004692602902650833,
  -0.0031840153969824314,
  0.02603411115705967,
  -0.010502157732844353,
  0.025949811562895775,
 

: 

# Retrieval

Vector search and similiarity between our prompt and the database

### Creating an search index and reranking

: 

: 

: 

: 

## Putting it all together to answer a question

In [ ]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://ragaoi.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

: 

: 

: 

: 

: 

In [ ]:
message_text = [
    {"role":"system","content":"You are an AI tutor who helps students with their homework and revision but doesn't give them answers to their problems"}, 
    {'role': 'user', 'content': 'What is the capital of Ireland?'}
    ]

completion = openai.ChatCompletion.create(
  engine="studentsaoi",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

: 

: 

: 

: 

: 

In [ ]:
completion.choices[0].message['content']

: 

: 

: 

: 

: 

## Testing and evaluation